<a href="https://colab.research.google.com/github/harshv47/r-india-Flair-Detector/blob/master/Jupyter%20Notebooks/Experiment_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###This log is divided into topics and in each topic the logs are sorted using time, so lower will have been written later in time.

# Importing
At first I imported common libraries.

These are used on every project so it's common.

Although this one has long texts, so maybe addional importing may be required

# Downloading the Dataset
To download the dataset, I used praw. However, it takes quite a while to download and I can't just press Ctrl + F8 to run all cells before anymore after every runtime restart.

So, to avoid the hassle I used a separate python file that is stored in */dataset/procure.py* to download data.

I have worked with praw before so I rehashed some previous code, also since this will go on GitHub and I might forget to remove sensitive details I have used a file to store the credential on local storage.

At first I downloaded the dataset by sorting r/india by hot. However, the dataset was really bias, I could not even get any examples for some flairs.

So, then I iterated and sorted by each flair. This was a better method.

I downloaded 100 examples for each flair.

There was an issue with the comments. Initially I did not want to include them because I thought it would be a hassle since some posts have thousands of comments it would be unwieldy to go through all that.

Later, I included comments because it was very valuable information for determining flairs. Eg. on many Political flairs one could see that many political party names and people were written in the comments and similarly, for Science flair ISRO and other Scientific orgs were routinely mentioned.

After running the models, I surmised that some models may be overfitting quite a lot. To counter this I increased the size of the dataset. Increasing the limit to 500.

# Importing the Dataset
The dataset was imported using *pd.read_csv*

I prefer quantitative data to representations, so I looked at shape, dtypes, etc.

The download was 100 examples for each flairs, and everyone had 100 except [R]eddiquette.

That might be a problem for that class, as the algorithm would naturally be less good at predicting for that class now.



Even though the limit is 500, the highest number of examples was only 248, meaning I have exhausted all the post for that flair type.

This would also mean that now classifiers will perform worse for [R]eddiquette class.

#Preproccesing and Modelling


The elements in body, url, author, title and comments are objects when they should be string

Wrote code to convert each element to string

Let's leave *url* *id* and *created* for a while, maybe I will add them in the future.

The body, title and comments will have to be cleaned and there are many uneccesary words and symbols

So, the code to clean and remove words in *body*, *title* and *comments* was written using the same apply technique

Train Test split was written. I used *X_test_cv* and *Y_test_cv* for cross-eval.

Score and Number of Comments feature is a number so it should be normalized. Wrote the code to normalized using mean, min and max.

Now, dealing with body, title, comments and author is difficult. They are all just strings.

I didn't knew a lot about TF-IDF so my first implementation was wrong which was problematic as took me reading quite a lot to understand that I messed up.

Finally after a lot of time was able to implement TF-IDF correctly.

Adding the feature columns thus produced to *X_train* directly, its quite a lot of features.

Implemented Random Forest, the accuracy is around 0.68.

Linear SVC takes a long time to give results and it has very low accuracy (around 0.1) that it very weird.

Tried, Logistic Regression it also has very low accuracy (around 0.09). That is very weird.

Tried, Decision Tree it is giving decent accuracy (around 0.58).

So, that means Tree based algorithms are giving good accuracy but why not svc.

Searched and Studied a lot, found many articles on this. Learned that SVM is not quite good at handling large sparse matrix. So, I converted the sparse matrix into a dense matrix before concatenating.

It seems that TF-IDF should not be used with *author* because the frequency for each author would be exceedingly low.
Therefore, removed author from cols_to_string.

Instead *author* can be used as categorical data, since it is very useful information. Redditors often post same kind of content in a subreddit. So, I can't really drop this feature.

Used *get_dummies* on *author* and concatenated them i *X_train*

Tried svc and LR again, the accuracy increased significantly (0.65 and 0.66), though still less than Random Forest which jumped to 0.72. 

Tried Naive Bias, it resulted in an error. The range of scores and number of comments is (-1,1). Converted the range to (0,1).

Naive Bias had the lowest accuracy at 0.52.

Realized that I could use domain names of the urls as a categorical data. This could reduce overfitting.

Used *tldextract* to extract domain name from *url*. Also applied *get_dummies*.

Tried applying ensemble, realized that predict_proba is needed for it. So, can't use in conjungtion with sgd as hinge loss doesn't have pedict_proba, same for SVC.

Learned that SVC does have predict_proba, enabled by parameter Probability=True. The accuracy of the ensemble is not that high.

Tried, with Random Froest and SGDClassifier the accuracy is less than Random Forest meaning it is actually pulling away from correct answer.

Finally decide to use CatBoost even though it might overfit.

On running CatBoost code, it gives an error that some columns have same names.

Searches for duplicates, found that there are a lot of them. Realize that maybe because tfidf, author and url are all just writing as the feature name with no prefixes. So, maybe someone may have mention a domain name or a redditor's name and it somehow got into tfidf as well thus there could be two columns.

Applied prefixes to *url* and *author* by using the *get_dummies* property *prefix*.

Applied the same for tfidf by changing the column vector that was passing as a parameter when converting to a DataFrame.

Finally ran all the models, accuracy increased a bit and the number of columns reduced.

CatBoost had the highest accuracy.